In [1]:
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from skimage import feature
from tqdm import tqdm

CHARACTERS = ["louie", "tommy", "andy", "ora"]
TRAIN_PATH = "C:/Users/Matei/Downloads/CAVA-2022-Tema2/antrenare/"
POSITIVE_PATH = "dataset/train/positives/"
NEGATIVE_PATH = "dataset/train/negatives/vertical/"

VALIDATION_PATH = "C:/Users/Matei/Downloads/CAVA-2022-Tema2/validare/Validare/"
VALIDATION_ANN = "C:/Users/Matei/Downloads/CAVA-2022-Tema2/validare/"
VALIDATION_P = "dataset/validation/positives/"
VALIDATION_N = "dataset/validation/negatives/"

# Extract faces from train

In [2]:
def get_annotations(character):
    annotations_file = open(TRAIN_PATH + character + "_annotations.txt")
    annotations = annotations_file.read().splitlines()
    annotations_file.close()

    return annotations

In [3]:
def is_valid_position(img, pos):
    if any(p < 0 for p in pos):
        return False
    if pos[0] >= img.shape[1] or pos[1] >= img.shape[1]:
        return False
    if pos[2] >= img.shape[0] or pos[2] >= img.shape[0]:
        return False
    return True

In [14]:
def extract_faces_from_annotations(character, source, positive_path, cutout=12):
    global count
    border = 0

    source += (character + "/")
    annotations = get_annotations(character)

    for annotation in tqdm(annotations):
         image_name, x_min, y_min, x_max, y_max, name = annotation.split()
         x_min, y_min, x_max, y_max = int(x_min), int(y_min), int(x_max), int(y_max)
         save_path = positive_path + name + "/"
         dict[name] += 1
         img = cv.imread(source + image_name)
         face = img[y_min:y_max, x_min:x_max, :]
         count += 1
         cv.imwrite(save_path + str(count) + ".jpg", face)

         pos = [x_min - cutout // 2, x_max + cutout // 2, y_min - cutout // 2, y_max + cutout // 2]
         if is_valid_position(img, pos):
            face = img[pos[2]:pos[3], pos[0]:pos[1], :]
            count += 1
            cv.imwrite(save_path + str(count) + ".jpg", face)

         if img.shape[0] >= 30 and img.shape[1] >= 30:
             pos = [x_min, x_max, y_min, y_max]
             pos[border % 4] += cutout
             if is_valid_position(img, pos):
                 face = img[pos[2]:pos[3], pos[0]:pos[1], :]
                 count += 1
                 cv.imwrite(save_path + str(count) + ".jpg", face)

             pos[border % 4] -= 2*cutout
             if is_valid_position(img, pos):
                 face = img[pos[2]:pos[3], pos[0]:pos[1], :]
                 count += 1
                 cv.imwrite(save_path + str(count) + ".jpg", face)

             if border % 4 == 0:
                 pos = [x_min - cutout // 2, x_max, y_min - cutout // 2, y_max]
                 if is_valid_position(img, pos):
                    face = img[pos[2]:pos[3], pos[0]:pos[1], :]
                    count += 1
                    cv.imwrite(save_path + str(count) + ".jpg", face)
             elif border % 4 == 1:
                 pos = [x_min + cutout // 2, x_max, y_min + cutout // 2, y_max]
                 if is_valid_position(img, pos):
                    face = img[pos[2]:pos[3], pos[0]:pos[1], :]
                    count += 1
                    cv.imwrite(save_path + str(count) + ".jpg", face)
             elif border % 4 == 2:
                 pos = [x_min, x_max + cutout // 2, y_min, y_max + cutout // 2]
                 if is_valid_position(img, pos):
                    face = img[pos[2]:pos[3], pos[0]:pos[1], :]
                    count += 1
                    cv.imwrite(save_path + str(count) + ".jpg", face)
             elif border % 4 == 3:
                 pos = [x_min, x_max + cutout // 2, y_min, y_max + cutout // 2]
                 if is_valid_position(img, pos):
                    face = img[pos[2]:pos[3], pos[0]:pos[1], :]
                    count += 1
                    cv.imwrite(save_path + str(count) + ".jpg", face)

         border += 1
    print(count)

In [15]:
count = 0
dict = {"andy": 0, "louie":0, "ora":0, "tommy":0, "unknown":0}
for character in CHARACTERS:
    extract_faces_from_annotations(character, TRAIN_PATH, POSITIVE_PATH)

100%|██████████| 1353/1353 [00:06<00:00, 198.19it/s]


6665


100%|██████████| 2576/2576 [00:10<00:00, 256.40it/s]


19218


100%|██████████| 1435/1435 [00:06<00:00, 222.54it/s]


26275


100%|██████████| 1872/1872 [00:07<00:00, 248.42it/s]

35479


In [13]:
for character in CHARACTERS:
    print(character, dict[character])
print(dict["unknown"])

louie 2042
tommy 1031
andy 1647
ora 1346
1170


# Analyze proportions for every character

In [ ]:
def compute_ratio(character, source):
    source += (character + "/")
    ratios = []

    for c in CHARACTERS:
        annotations = get_annotations(c)
        for annotation in annotations:
            image_name, x_min, y_min, x_max, y_max, name = annotation.split()
            x_min, y_min, x_max, y_max = int(x_min), int(y_min), int(x_max), int(y_max)
            if name == character:
                width = x_max - x_min + 1
                height = y_max - y_min + 1
                ratios.append(height/width)

    return sum(ratios) / len(ratios)


In [ ]:
for character in CHARACTERS:
    print(character + ":", compute_ratio(character, TRAIN_PATH))
print("unknown:", compute_ratio("unknown", TRAIN_PATH))

# width/height
# louie: 1.251815758043986
# tommy: 1.3285152584256952
# andy: 0.6870738648331448
# ora: 0.7801411367617421
# unknown: 0.9044525116339853

# height/width
# louie: 0.814094476628571
# tommy: 0.7733086598193187
# andy: 1.4993275758468239
# ora: 1.2960066470770064
# unknown: 1.1593202925732515

# Extract negative examples

In [ ]:
def intersection_over_union(box_a, box_b):
    x_min = max(box_a[0], box_b[0])
    y_min = max(box_a[1], box_b[1])
    x_max = min(box_a[2], box_b[2])
    y_max = min(box_a[3], box_b[3])

    inter_area = max(0, x_max - x_min + 1) * max(0, y_max - y_min + 1)

    box_a_area = (box_a[2] - box_a[0] + 1) * (box_a[3] - box_a[1] + 1)
    box_b_area = (box_b[2] - box_b[0] + 1) * (box_b[3] - box_b[1] + 1)

    iou = inter_area / float(box_a_area + box_b_area - inter_area)

    return iou

In [ ]:
def get_illegal_fields(annotations, current_image):
     illegal_fields = set()
     for annotation in annotations:
        image_name, x_min, y_min, x_max, y_max, name = annotation.split()
        if image_name == current_image:
            zone = (int(x_min), int(y_min), int(x_max), int(y_max))
            illegal_fields.add(zone)
     return illegal_fields

In [ ]:
def distant_rand(a, b, x, d):
    lb = max(a, x - d + 1)
    ub = min(b, x + d - 1)
    k = ub - lb + 1

    if b-k < a:
        return None
    else:
        y = np.random.randint(a, b - k)
        if y > x - d:
            y = y + k
        return y

In [ ]:
def extract_negative_examples(character, save_path, l=60, L=72, num_samples=28, iou_thresh=0.15, limit=100000):
    global count

    annotations = get_annotations(character)
    used_files = set()

    for annotation in tqdm(annotations):
        image_name = annotation.split()[0]
        if image_name not in used_files:
            used_files.add(image_name)
            illegal_fields = get_illegal_fields(annotations, image_name)

            img = cv.imread(TRAIN_PATH + character + "/" + image_name)
            num_cols = img.shape[0]
            num_rows = img.shape[1]

            point = (np.random.randint(low=0, high=num_rows - l), np.random.randint(low=0, high=num_cols - L))
            for i in range(num_samples):
                x = distant_rand(0, num_rows - l, point[0], 3 * l // 2)
                y = distant_rand(0, num_cols - L, point[1], 3 * L // 2)
                point = (x, y)

                square = img[y: y + L, x: x + l]
                scores = [intersection_over_union((x, y, x + l,  y + L), zone)
                      for zone in illegal_fields]

                if all(score <= iou_thresh for score in scores):
                    cv.imwrite(save_path + str(count) + ".jpg" , square)
                    count += 1

                if count > limit: return

In [ ]:
count = 0
for character in CHARACTERS:
    extract_negative_examples(character, NEGATIVE_PATH)

# Extract positives from validation

In [ ]:
def get_positives_validation():
    annotations_file = open(VALIDATION_ANN + "validare_annotations.txt")
    annotations = annotations_file.read().splitlines()
    annotations_file.close()

    count = 0
    for annotation in tqdm(annotations):
         image_name, x_min, y_min, x_max, y_max, name = annotation.split()
         x_min, y_min, x_max, y_max = int(x_min), int(y_min), int(x_max), int(y_max)

         img = cv.imread(VALIDATION_PATH + image_name)
         face = img[y_min:y_max, x_min:x_max, :]
         count += 1
         cv.imwrite(VALIDATION_P + str(count) + ".jpg", face)

In [ ]:
get_positives_validation()

# Extract negative examples

In [ ]:
def get_negatives_validation(l=64, num_samples=5, iou_thresh=0.25, limit=1000):
    annotations_file = open(VALIDATION_ANN + "validare_annotations.txt")
    annotations = annotations_file.read().splitlines()
    annotations_file.close()

    used_files = set()
    count = 0

    for annotation in tqdm(annotations):
        image_name = annotation.split()[0]
        if image_name not in used_files:
            used_files.add(image_name)
            illegal_fields = get_illegal_fields(annotations, image_name)

            img = cv.imread(TRAIN_PATH + character + "/" + image_name)
            num_cols = img.shape[0]
            num_rows = img.shape[1]

            point = (np.random.randint(low=0, high=num_rows - l), np.random.randint(low=0, high=num_cols - l))
            for i in range(num_samples):
                x = distant_rand(0, num_rows - l, point[0], 2 * l)
                y = distant_rand(0, num_cols - l, point[1], 2 * l)
                point = (x, y)

                square = img[y: y + l, x: x + l]
                scores = [intersection_over_union((x, y, x + l,  y + l), zone)
                      for zone in illegal_fields]

                if all(score <= iou_thresh for score in scores):
                    cv.imwrite(VALIDATION_N + str(count) + ".jpg" , square)
                    count += 1

                if count > limit: return

In [ ]:
get_negatives_validation()

In [40]:
def get_more_from_character(character, source, positive_path, cutout=15):
    global count
    border = 0

    source += (character + "/")
    annotations = get_annotations(character)


    for annotation in tqdm(annotations):
        image_name, x_min, y_min, x_max, y_max, name = annotation.split()
        x_min, y_min, x_max, y_max = int(x_min), int(y_min), int(x_max), int(y_max)

        if name != "unkonwn":
            save_path = positive_path + name + "/"
            img = cv.imread(source + image_name)
            face = img[y_min:y_max, x_min:x_max, :]
            count += 1
            cv.imwrite(save_path + str(count) + ".jpg", face)

            pos = [x_min - cutout, x_max + cutout, y_min - cutout, y_max + cutout]
            if is_valid_position(img, pos):
                face = img[pos[2]:pos[3], pos[0]:pos[1], :]
                count += 1
                cv.imwrite(save_path + str(count) + ".jpg", face)

            if img.shape[0] >= 30 and img.shape[1] >= 30:
                pos = [x_min, x_max, y_min, y_max]
                pos[border % 4] += cutout
                if is_valid_position(img, pos):
                    face = img[pos[2]:pos[3], pos[0]:pos[1], :]
                    count += 1
                    cv.imwrite(save_path + str(count) + ".jpg", face)

                pos[border % 4] -= 2*cutout
                if is_valid_position(img, pos):
                    face = img[pos[2]:pos[3], pos[0]:pos[1], :]
                    count += 1
                    cv.imwrite(save_path + str(count) + ".jpg", face)

            if border % 2 == 0:
                pos = [x_min - cutout // 2, x_max, y_min - cutout // 2, y_max]
                if is_valid_position(img, pos):
                    face = img[pos[2]:pos[3], pos[0]:pos[1], :]
                    count += 1
                    cv.imwrite(save_path + str(count) + ".jpg", face)

                    pos = [x_min + cutout // 2, x_max, y_min + cutout // 2, y_max]
                if is_valid_position(img, pos):
                    face = img[pos[2]:pos[3], pos[0]:pos[1], :]
                    count += 1
                    cv.imwrite(save_path + str(count) + ".jpg", face)
            elif border % 2 == 1:
                pos = [x_min, x_max + cutout // 2, y_min, y_max + cutout // 2]
                if is_valid_position(img, pos):
                    face = img[pos[2]:pos[3], pos[0]:pos[1], :]
                    count += 1
                    cv.imwrite(save_path + str(count) + ".jpg", face)

                pos = [x_min, x_max + cutout // 2, y_min, y_max + cutout // 2]
                if is_valid_position(img, pos):
                    face = img[pos[2]:pos[3], pos[0]:pos[1], :]
                    count += 1
                    cv.imwrite(save_path + str(count) + ".jpg", face)

            l = x_max - x_min + 1
            L = y_max - y_min + 1
            l = max(L, l)
            face = img[y_min:y_min + l, x_min:x_min + l]
            count += 1
            cv.imwrite(save_path + str(count) + ".jpg", face)

        border += 1